In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data URL Google Sheets (CSV export)
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSlNZcTx3HOD2HhZ_nXgS5gqlDCRSzY1VwhYh059gaLXnFfNM48sPS24L2O67FpvsSU2mKDxAJcKDUY/pub?output=csv"

In [ ]:
# Load data
df = pd.read_csv(url)

In [ ]:
print(df)

In [ ]:
df["last_transaksi"] = pd.to_datetime(
    df["last_transaksi"],
    format="%d/%m/%Y"
)

In [ ]:
# Tanggal referensi (akhir bulan otomatis)
reference_date = pd.Timestamp.today().to_period("M").to_timestamp("M")

In [ ]:
# Hitung bulan tidak aktif
df["months_inactive"] = (
    (reference_date.year - df["last_transaksi"].dt.year) * 12 +
    (reference_date.month - df["last_transaksi"].dt.month)
)

In [ ]:
# Status churn
df["churn_status"] = df["months_inactive"].apply(
    lambda x: "Churn" if x >= 3 else "Active"
)

In [ ]:

print(df.head())

In [ ]:
# Segmentasi + Complaint
df["churn_segment"] = df["months_inactive"].apply(
    lambda x: "High Risk Churn" if x >= 6 else
              "Churn" if x >= 3 else
              "Active"
)
print(df.head())

In [ ]:
import matplotlib.pyplot as plt

# Ringkasan jumlah customer per status
churn_summary = (
    df["churn_status"]
    .value_counts()
    .reset_index()
)

churn_summary.columns = ["churn_status", "total_customer"]

# Buat grafik
plt.figure()
plt.bar(
    churn_summary["churn_status"],
    churn_summary["total_customer"]
)
plt.xlabel("Status Customer")
plt.ylabel("Jumlah Customer")
plt.title("Monitoring Churn Customer (≥ 3 Bulan Tidak Transaksi)")
plt.show()


In [ ]:
segment_summary = (
    df["churn_segment"]
    .value_counts()
    .reset_index()
)

segment_summary.columns = ["churn_segment", "total_customer"]

plt.figure()
plt.bar(
    segment_summary["churn_segment"],
    segment_summary["total_customer"]
)
plt.xlabel("Segment Churn")
plt.ylabel("Jumlah Customer")
plt.title("Distribusi Segment Churn Customer")
plt.show()


In [ ]:
cx_summary = (
    df.groupby(["churn_status", "complaint_flag"])
    .size()
    .reset_index(name="total_customer")
)

for status in cx_summary["churn_status"].unique():
    subset = cx_summary[cx_summary["churn_status"] == status]
    plt.figure()
    plt.bar(
        subset["complaint_flag"].astype(str),
        subset["total_customer"]
    )
    plt.xlabel("Complaint Flag (0 = No, 1 = Yes)")
    plt.ylabel("Jumlah Customer")
    plt.title(f"Complaint vs Churn Status: {status}")
    plt.show()

In [ ]:
plt.figure()
plt.hist(df["months_inactive"])
plt.xlabel("Jumlah Bulan Tidak Transaksi")
plt.ylabel("Jumlah Customer")
plt.title("Distribusi Lama Customer Tidak Bertransaksi")
plt.show()


In [ ]:
df["early_warning"] = df["months_inactive"].apply(
    lambda x: "Warning (2 Bulan)" if x == 2 else
              "Normal" if x < 2 else
              "Churn"
)

warning_summary = (
    df["early_warning"]
    .value_counts()
    .reset_index()
)
warning_summary.columns = ["status", "total_customer"]

plt.figure()
plt.bar(
    warning_summary["status"],
    warning_summary["total_customer"]
)
plt.xlabel("Status Customer")
plt.ylabel("Jumlah Customer")
plt.title("Early Warning Churn Monitoring")
plt.show()
